In [1]:
import pandas as pd
import numpy as np
import pycountry
import re

companies=pd.read_csv('company.csv',delimiter='\t')
rounds2=pd.read_csv('rounds2.csv',encoding='iso-8859-1')
mapping=pd.read_csv('mapping.csv',encoding='iso-8859-1')

    
companies.permalink=companies.permalink.apply(lambda x:str.lower(x))
rounds2.company_permalink=rounds2.company_permalink.apply(lambda x:str.lower(x))



In [2]:
print(companies.info(),rounds2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB

In [3]:
print("# of unique companies in rounds2: ",rounds2.company_permalink.unique().shape[0])

# of unique companies in rounds2:  66370


In [4]:
print("# of unique companies in companies: ",companies.permalink.unique().shape[0])

# of unique companies in companies:  66368


In [5]:
master_frame=pd.merge(rounds2,companies,how='left',left_on='company_permalink',right_on='permalink')
print(master_frame.shape)

(114949, 16)


In [6]:
print("*****pecentage of missing values**** \n \n",(master_frame.isnull().sum()/master_frame.shape[0])*100)

*****pecentage of missing values**** 
 
 company_permalink           0.000000
funding_round_permalink     0.000000
funding_round_type          0.000000
funding_round_code         72.909725
funded_at                   0.000000
raised_amount_usd          17.390321
permalink                   0.027838
name                        0.028708
homepage_url                5.359768
category_list               2.984802
status                      0.027838
country_code                7.556395
state_code                  9.528574
region                      8.851752
city                        8.849142
founded_at                 17.870534
dtype: float64


In [7]:
master_frame=master_frame[~master_frame.raised_amount_usd.isnull()]
print("*****pecentage of missing values**** \n \n",(master_frame.isnull().sum()/master_frame.shape[0])*100)

*****pecentage of missing values**** 
 
 company_permalink           0.000000
funding_round_permalink     0.000000
funding_round_type          0.000000
funding_round_code         70.335619
funded_at                   0.000000
raised_amount_usd           0.000000
permalink                   0.017902
name                        0.018956
homepage_url                4.578818
category_list               1.114165
status                      0.017902
country_code                6.167925
state_code                  8.012932
region                      7.428469
city                        7.425310
founded_at                 16.824103
dtype: float64


In [8]:
master_frame = master_frame.drop('funding_round_code', axis=1)
master_frame = master_frame.drop('founded_at', axis=1)
master_frame = master_frame.drop('homepage_url', axis=1)

In [9]:
print("*****pecentage of missing values**** \n \n",(master_frame.isnull().sum()/master_frame.shape[0])*100)

*****pecentage of missing values**** 
 
 company_permalink          0.000000
funding_round_permalink    0.000000
funding_round_type         0.000000
funded_at                  0.000000
raised_amount_usd          0.000000
permalink                  0.017902
name                       0.018956
category_list              1.114165
status                     0.017902
country_code               6.167925
state_code                 8.012932
region                     7.428469
city                       7.425310
dtype: float64


In [10]:
suitable=[]
for i in master_frame.funding_round_type.unique():
    average= master_frame.loc[master_frame.funding_round_type==i]
    average=average.raised_amount_usd.mean()/1000000
    print(i,'-------------------',average,"millions")
    if  round(average)<15 and round(average)>5:
        suitable.append(i)
        
print("\n\n******most suitable type of investment for sparkfunds****** :",suitable)

venture ------------------- 11.748949129489528 millions
seed ------------------- 0.7198179969071729 millions
undisclosed ------------------- 19.242370225663716 millions
convertible_note ------------------- 1.4534385431928127 millions
private_equity ------------------- 73.30859302944215 millions
debt_financing ------------------- 17.043526023046994 millions
angel ------------------- 0.9586944697530865 millions
grant ------------------- 4.30057633945386 millions
equity_crowdfunding ------------------- 0.5383682109120521 millions
post_ipo_equity ------------------- 82.1824938710191 millions
post_ipo_debt ------------------- 168.70457182236842 millions
product_crowdfunding ------------------- 1.3631310699481867 millions
secondary_market ------------------- 79.6496301 millions
non_equity_assistance ------------------- 0.4112030547945206 millions


******most suitable type of investment for sparkfunds****** : ['venture']


In [11]:
#checkpoint from countries
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_territorial_entities_where_English_is_an_official_language")

def clean(name):
    end=name.find('[')
    if end==-1:
        return name
    if end > 0:
        return name[:end]


def name_code(input_name):
    country_dict = {}
    for country in pycountry.countries:
            country_dict[str.lower(country.name)] = country.alpha_3
    try:
        return country_dict[str.lower(input_name)]
    except KeyError:
        return 'unknown'
    
table=tables[1]
countries=pd.DataFrame(table)
countries.columns = countries.iloc[0]
countries=countries.drop(countries.index[0])

countries.Country=countries.Country.apply( lambda x:clean(x))
countries.Country=countries.Country.apply( lambda x:clean(x))
countries=countries.drop('Nr',axis=1)
countries.head()

df=pd.DataFrame(tables[0])
df.columns=df.iloc[0]
df=df.drop(df.index[0])
df.Country=df.Country.apply(lambda x:clean(x))
df=df.drop('Nr',axis=1)

countries=pd.concat([df,countries],axis=0)
 
countries.Country=countries.Country.apply(lambda x:name_code(x))
eng_countries=countries.Country.tolist()
countries.head()

,Alpha-3 Code,Country,Population,Population1,Primary language?,Region
1,AUS,AUS,25019600,NaN,Yes,Oceania
2,NZL,NZL,4893830,NaN,Yes,Oceania
3,GBR,GBR,66040229,NaN,Yes,Europe
4,USA,USA,325719178,NaN,Yes,North America/Oceania
1,ATG,ATG,NaN,85000,Yes,Caribbean


In [12]:
venture_df = master_frame[master_frame["funding_round_type"]==suitable[0]]
venture_by_country = venture_df.groupby('country_code')
venture_by_country['raised_amount_usd'].sum().sort_values(ascending=False).head(9)

top9 = venture_df[venture_df['country_code'].isin(["USA","CHN","GBR","IND","CAN","FRA","ISR","DEU","JPN"])]

top9=top9[~top9.category_list.isnull()]

top9['primary_sector'] =top9['category_list'].astype(str).apply(lambda x: x.split('|')[0])
top9.shape

mapping=mapping[~mapping.category_list.isnull()]
mapping.head()


mapping['category_list'] = mapping.category_list.apply(lambda x: re.sub('[0]', 'na', str(x)))

mapping=pd.melt(mapping, id_vars=['category_list'], var_name=['main_sector'])
mapping=mapping[mapping.value==1]
mapping=mapping.drop('value',axis=1)
mapping.shape
top9=pd.merge(top9,mapping,how="left",left_on="primary_sector",right_on="category_list")
top9.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list_x,status,country_code,state_code,region,city,primary_sector,category_list_y,main_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,Media,operating,IND,16,Mumbai,Mumbai,Media,Media,Entertainment
1,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,Curated Web,Curated Web,"News, Search and Messaging"
2,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,Biotechnology,Biotechnology,Cleantech / Semiconductors
3,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,09-11-2015,20000000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,CA,SF Bay Area,Mountain View,Analytics,Analytics,"Social, Finance, Analytics, Advertising"
4,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,03-01-2013,1700000.0,/organization/0xdata,H2O.ai,Analytics,operating,USA,CA,SF Bay Area,Mountain View,Analytics,Analytics,"Social, Finance, Analytics, Advertising"


In [13]:
pos=1
top3=[]
for i in top_list:
    if pos == 4:
        break
    if i in eng_countries:
        top3.append(i)
        print(pos,i)
        pos+=1

print(top3)

NameError: name 'top_list' is not defined

In [ ]:
# lets drop all rows whoes investment is not between 5 and 15 million
top9 = top9.drop(top9[(top9.raised_amount_usd < 5000000)].index)
top9 = top9.drop(top9[(top9.raised_amount_usd > 15000000)].index)

d1 = top9[top9['country_code'] == 'USA']
d2 = top9[top9['country_code'] == 'GBR']
d3 = top9[top9['country_code'] == 'IND']

In [ ]:
d1.